In [1]:

from pyspark import SparkConf
from pyspark.sql import SparkSession
from operator import add
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.119:7077") \
        .appName("adam-axelsson")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/23 18:12:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/23 18:12:09 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


In [2]:
swe = spark_context.textFile("hdfs://192.168.2.119:9000/europarl/europarl-v7.sv-en.sv")
eng = spark_context.textFile("hdfs://192.168.2.119:9000/europarl/europarl-v7.sv-en.en")

#A.1.1
print("Line count Swedish: %d" % swe.count())
#A.1.2
print("Line count English: %d" % eng.count())

#A.1.3
assert(swe.count() == eng.count())

#A.1.4
totalPartitions = swe.getNumPartitions() + eng.getNumPartitions()
print("Total number of Partitions: %d" % totalPartitions)

Line count Swedish: 1862234


Line count English: 1862234


Total number of Partitions: 5


In [3]:
#A.2.1
def lowercase_split(lines):
    lower_case = lines.map(lambda line: line.lower())
    lines_split = lower_case.map(lambda line: line.split(' '))
    return lines_split

In [4]:
#A.2.2
swe_split = lowercase_split(swe)
eng_split = lowercase_split(eng)

#A.2.3
print(swe_split.take(10))
print(eng_split.take(10))

[['återupptagande', 'av', 'sessionen'], ['jag', 'förklarar', 'europaparlamentets', 'session', 'återupptagen', 'efter', 'avbrottet', 'den', '17', 'december.', 'jag', 'vill', 'på', 'nytt', 'önska', 'er', 'ett', 'gott', 'nytt', 'år', 'och', 'jag', 'hoppas', 'att', 'ni', 'haft', 'en', 'trevlig', 'semester.'], ['som', 'ni', 'kunnat', 'konstatera', 'ägde', '"den', 'stora', 'år', '2000-buggen"', 'aldrig', 'rum.', 'däremot', 'har', 'invånarna', 'i', 'ett', 'antal', 'av', 'våra', 'medlemsländer', 'drabbats', 'av', 'naturkatastrofer', 'som', 'verkligen', 'varit', 'förskräckliga.'], ['ni', 'har', 'begärt', 'en', 'debatt', 'i', 'ämnet', 'under', 'sammanträdesperiodens', 'kommande', 'dagar.'], ['till', 'dess', 'vill', 'jag', 'att', 'vi,', 'som', 'ett', 'antal', 'kolleger', 'begärt,', 'håller', 'en', 'tyst', 'minut', 'för', 'offren', 'för', 'bl.a.', 'stormarna', 'i', 'de', 'länder', 'i', 'europeiska', 'unionen', 'som', 'drabbats.'], ['jag', 'ber', 'er', 'resa', 'er', 'för', 'en', 'tyst', 'minut.'], 

In [5]:

def word_count(word_list):
    flattened_list = word_list.flatMap(lambda list: [line for line in list])
    stripped_list = flattened_list.map(lambda word: word.strip('.,- '))
    #Maybe remove?
    mapper = stripped_list.map(lambda word: (word, 1))
    reducer = mapper.reduceByKey(add)
    return reducer

In [6]:

#A.3.1

word_count_eng = word_count(eng_split)
word_count_eng_sorted = word_count_eng.sortBy(lambda x: -x[1])

word_count_swe = word_count(swe_split)
word_count_swe_sorted = word_count_swe.sortBy(lambda x: -x[1])

print("Most common english words: ")
print(word_count_eng_sorted.take(10))
print("\n\n")
print("Most common swedish words: ")
print(word_count_swe_sorted.take(10))


#A.3.2
"""
Results seem reasonable, according to wikipedia these are the most common english words:
the, be, to, of, a, and, in, that, have, i. This seem to coincide quite well with the results


For swedish the 10 most common words according to wikipedia is:
i, och, att, det, som, en, på, är, av, för. All these words are in our most common words except for på which is 
replaced by för
"""

Most common english words: 
[('the', 3498502), ('of', 1661744), ('to', 1543055), ('and', 1317854), ('in', 1088052), ('that', 837711), ('a', 773672), ('is', 773643), ('for', 537364), ('we', 525767)]



Most common swedish words: 
[('att', 1709436), ('och', 1349914), ('i', 1053572), ('det', 949714), ('som', 917069), ('för', 914357), ('av', 740524), ('är', 700669), ('en', 620774), ('vi', 545269)]


'\nResults seem reasonable, according to wikipedia these are the most common english words:\nthe, be, to, of, a, and, in, that, have, i. This seem to coincide quite well with the results\n\n\nFor swedish the 10 most common words according to wikipedia is:\ni, och, att, det, som, en, på, är, av, för. All these words are in our most common words except for på which is \nreplaced by för\n'

In [7]:
word_count_eng_sorted = word_count_eng.sortBy(lambda x: -x[1])
word_count_swe_sorted = word_count_swe.sortBy(lambda x: -x[1])
print("Most common english words: ")
print(word_count_eng_sorted.take(10))
print("\n\n")
print("Most common swedish words: ")
print(word_count_swe_sorted.take(10))


Most common english words: 
[('the', 3498502), ('of', 1661744), ('to', 1543055), ('and', 1317854), ('in', 1088052), ('that', 837711), ('a', 773672), ('is', 773643), ('for', 537364), ('we', 525767)]



Most common swedish words: 
[('att', 1709436), ('och', 1349914), ('i', 1053572), ('det', 949714), ('som', 917069), ('för', 914357), ('av', 740524), ('är', 700669), ('en', 620774), ('vi', 545269)]


In [8]:
#A.4.1
sv1 = swe_split.zipWithIndex()
en1 = eng_split.zipWithIndex()


In [9]:
#A.4.2
sv2 = sv1.map(lambda x: (x[1],x[0]))
en2 = en1.map(lambda x: (x[1],x[0]))

In [10]:
#A.4.3
join3 = en2.join(sv2)
print(join3.take(1))

[(902130, (['written', 'statements', '(rule', '142)'], ['skriftliga', 'förklaringar', '(artikel', '142)']))]


In [11]:
#A.4.4
join4 = join3.filter(lambda x: x[1][1] != "").filter(lambda x: x[1][1] is not None)
join5 = join4.filter(lambda x: x[1][0] != "").filter(lambda x: x[1][0] is not None)
print(join4.count())
print(join3.count())
print(join5.count())

#Bad filtering or no missing values?

1862234


1862234


1862234


In [12]:
#A.4.5
join6 = join5.filter(lambda x: len(x[1][0]) <= 5)

print(join6.count())

85439


In [13]:
#A.4.6
join7 = join6.filter(lambda x: len(x[1][0]) == len(x[1][1]))
print(join7.count())

38016


In [14]:
#A.4.7
join8 = join7.flatMap(lambda x: zip(x[1][0], x[1][1]))
for obj in join8.take(10):
    print(list(obj))
    
#Fungerar på vissa meningar, men inte alla

['is', 'herr']
['that', 'brok,']
['right,', 'är']
['mr', 'det']
['brok?', 'så?']
['peace', 'freden']
['is', 'är']
['necessary.', 'nödvändig.']
['we', 'det']
['know', 'vet']


In [15]:
#A.4.8

join9 = join8.map(lambda x: (x, 1))
join10 = join9.reduceByKey(add)

for obj in join10.take(10):
    print(obj)

(('fearful.', 'rädda.'), 1)
(('closed.', 'avslutad.'), 2951)
(('why?', 'varför?'), 374)
(('committing', 'in'), 1)
(('more', 'mer'), 69)
(('am', 'gläder'), 4)
(('very', 'mig'), 7)
(('the', 'ärade'), 4)
(('27.', '27.'), 14)
(('this', 'det'), 669)


In [16]:
#A.4.9
join11 = join10.sortBy(lambda x: -x[1])

print(join11.take(10))

[(('is', 'är'), 4699), (('closed.', 'avslutad.'), 2951), (('(applause)', '(applåder)'), 2546), (('', '.'), 2223), (('.', '.'), 2084), (('that', 'det'), 1494), (('we', 'vi'), 1443), (('the', 'jag'), 1336), (('is', 'debatten'), 1327), (('debate', 'förklarar'), 1319)]


In [17]:
spark_session.stop()